# Imports

In [ ]:
from elasticsearch_dsl import Search, Q, Range, connections
from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
#from openai import OpenAI

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

In [ ]:
_ES_ENDPOINTS = {
    "prod": {"host": "ha-proxy-elasticsearch", "port": 9200},
    "test": {"host": "test-elasticsearch", "port": 9200},
}
ES_INDEX = "merged_events"

In [ ]:
connections.create_connection(alias="ProductionEnvironment", hosts="http://ha-proxy-elasticsearch:9200", timeout= None)

# meslis

In [ ]:
event_search = Search(using="ProductionEnvironment", index="merged_events")

In [ ]:
city_query_Dresden = (Q("match", mergedLocation__address__city="Dresden")   )
city_query_Freiburg = (Q("match", mergedLocation__address__city="Freiburg")   ) # does this include work inclusion for "Freiburg am Breisgau", etc. - yes it does
date_query_DD  =  Q({"range": {"startDate": { "gte": "2024-01-01","lt": "2024-04-01"}}})
date_query_FB  =  Q({"range": {"startDate": { "gte": "2023-06-01","lt": "2023-08-01"}}})
date_query_both  =  Q({"range": {"startDate": { "gte": "2024-09-03","lt": "2024-11-01"}}})

In [ ]:
event_filter = event_search.query(
    ((city_query_Dresden | city_query_Freiburg) & date_query_both) | \
    (city_query_Dresden & date_query_DD) | \
    (city_query_Freiburg & date_query_FB)
)
event_filter.count()

In [ ]:
# names of postal codes
# gemaNo_address_postalCode
# mergedLocation_address_postalCode

In [ ]:
meslis_events = []

for hit in tqdm(event_filter.scan(), total = event_filter.count() ):
    meslis_events.append(hit.to_dict())
    

In [ ]:
len(meslis_events)

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    """
    Recursively flatten a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            # Recursively flatten nested dictionaries
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # Handle lists: convert to string (comma-separated)
            items.append((new_key, ', '.join(map(str, v))))
        else:
            items.append((new_key, v))
    return dict(items)


In [ ]:
flat_meslis_events= [flatten_dict(event) for event in meslis_events]

In [ ]:
df_meslis = pd.DataFrame(flat_meslis_events)

In [ ]:
df_meslis.mergedLocation_address_city.unique()

In [ ]:
df_meslis.columns

In [ ]:
len(df_meslis)

In [ ]:
df_meslis[['startDate', 'id']].head()

In [ ]:
df_meslis[['startDate', 'id']] [df_meslis.startDate.str.endswith("30:00Z")]

In [ ]:
sorted(df_meslis.startDate.str[:10].unique().tolist())

In [ ]:
df_meslis.mergedLocation_address_city.unique()

In [ ]:
df_meslis = df_meslis[~df_meslis.mergedLocation_address_city.isin(["Freiburg/Elbe", "Freiburg (Elbe)"])]

In [ ]:
sorted(df_meslis.mergedLocation_address_postalCode.unique().tolist())

In [ ]:
df_meslis = df_meslis[~(df_meslis.mergedLocation_address_postalCode=="21729")]

In [ ]:
len(df_meslis)

In [ ]:
df_meslis.mergedLocation_address_city.unique()

In [ ]:
df_meslis.mergedLocation_address_city = df_meslis.mergedLocation_address_city.str.replace(", Germany", "").str.replace("-Neustadt", "")

In [ ]:
df_meslis.to_pickle(f"../data/events/meslis/df_meslis {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")

# mns

In [ ]:
event_search = Search(using="ProductionEnvironment", index="gema_nutzungsfaelle_v5")

In [ ]:
city_query_Dresden = (Q("match", gemaNo__address__city="Dresden")   )
city_query_Freiburg = (Q("match", gemaNo__address__city="Freiburg")   )
date_query_DD  =  Q({"range": {"startDate": { "gte": "2024-01-01","lt": "2024-04-01"}}})
date_query_FB  =  Q({"range": {"startDate": { "gte": "2023-06-01","lt": "2023-08-01"}}})
date_query_both =  Q({"range": {"startDate": { "gte": "2024-09-03","lt": "2024-11-01"}}})

In [ ]:
event_filter = event_search.query(
    ((city_query_Dresden | city_query_Freiburg) & date_query_both) | \
    (city_query_Dresden & date_query_DD) | \
    (city_query_Freiburg & date_query_FB)
)
event_filter.count()

In [ ]:
gema_events = []

for hit in tqdm(event_filter.scan(), total = event_filter.count() ):
    gema_events.append(hit.to_dict())
    

In [ ]:
len(gema_events)

In [ ]:
flat_gema_events= [flatten_dict(event) for event in gema_events]

In [ ]:
df_gema = pd.DataFrame(flat_gema_events)

In [ ]:
df_gema.gemaNo_address_city.unique()

In [ ]:
df_gema = df_gema[~ df_gema.gemaNo_address_city.isin(["Freiburg (Elbe)"])]

In [ ]:
# check that postal code is of Freiburg im Breisgau
df_gema.loc[df_gema.gemaNo_address_city.str.contains("Freiburg"), "gemaNo_address_postalCode"].unique()

In [ ]:
#make sure the date filter is correct
sorted(df_gema.startDate.str[:10].unique().tolist())

In [ ]:
len(df_gema)

In [ ]:
df_gema.dropna(subset="gemaNo_address_coordinate_lat").gemaNo_address_coordinate_lat

In [ ]:
df_gema.dropna(subset="gemaNo_address_coordinate_lat")[["gemaNo_address_street", "gemaNo_address_streetHouseNumber"]].head()

In [ ]:
#df_gema.gemaNo_address_coordinate_lat.unique()

In [ ]:
#check postal codes
sorted(df_gema.gemaNo_address_postalCode.unique().tolist())

In [ ]:
df_gema.to_pickle(f"../data/events/gema/df_gema {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")

# Apify

In [ ]:
os.walk("Downloads")

In [ ]:
os.getcwd()

In [ ]:
json_files = []
for root, dirs, files in os.walk('C:\\Users\\valentyna.sinichenko\\Downloads'):
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))


In [ ]:
all_json_data = []
known_source_urls = set()

In [ ]:
processed_files = []

for file_path in tqdm(json_files, total = len(json_files)):
    try:
        with open(file_path, "r", encoding="utf8") as f:
            data = json.load(f)
        if "apifyParsingAddressLogic" in data[0].keys():
            continue
        if "event" not in data[0].keys():
            continue
    
        for record in tqdm(data, total = len(data)):
            cur_source_url = record["event"]["sourceInformation"]["sourceUrl"]
            if cur_source_url not in known_source_urls:
                all_json_data.append(record)
                known_source_urls.add(cur_source_url)
    
        print(f"{len(all_json_data)=}")
        processed_files.append(file_path)
    except Exception as e:
        print(f"{e=}")
        print(file_path.upper())
        
    

In [ ]:
not_processed_files = [file for file in json_files if file not in processed_files]

In [ ]:
not_processed_files

In [ ]:
not_processed_files = [file for file in not_processed_files if "address-parser" not in file]

In [ ]:
not_processed_files = [file for file in not_processed_files if "Main" not in file]

In [ ]:
not_processed_files = [file for file in not_processed_files if "google-maps" not in file]

In [ ]:
for file_path in tqdm(not_processed_files, total = len(not_processed_files)):
    try:
        with open(file_path, "r") as f:
            data = json.load(f)
        if "apifyParsingAddressLogic" in data[0].keys():
            continue
        if "event" not in data[0].keys():
            continue
    
        for record in tqdm(data, total = len(data)):
            cur_source_url = record["event"]["sourceInformation"]["sourceUrl"]
            if cur_source_url not in known_source_urls:
                all_json_data.append(record)
                known_source_urls.add(cur_source_url)
    
        print(f"{len(all_json_data)=}")
        processed_files.append(file_path)
    except Exception as e:
        print(f"{e=}")
        print(file_path.upper())
        
    

In [ ]:
not_processed_files

## keep only relevant cities

In [ ]:
all_json_data[0]["location"]["address"]["city"]

In [ ]:
all_json_data[0]["location"]["address"]["rawAddress"]

In [ ]:
{}.get("smth")

In [ ]:
all_relevant_location_data = []
cities = []

for record in tqdm(all_json_data):
    try:    
        city = record.get("location", {}).get("address", {}).get("city", "")
    except Exception as e:
        print(e)

        continue

    try:
        raw_address = record.get("location", {}).get("address", {}).get("rawAddress", "")
    except Exception as e:
        print(e)

        continue
        

    if city:
        if "Dresden" in city or "Freiburg" in city:
            print(city)
            all_relevant_location_data.append(record)
            cities.append(city)
            continue
    if raw_address:
        if "Dresden" in raw_address or "Freiburg" in raw_address:
            print(raw_address)
            all_relevant_location_data.append(record)
            continue
            
    
    
    

In [ ]:
set(cities)

In [ ]:
len(all_relevant_location_data)

In [ ]:
pd.to_datetime(all_relevant_location_data[200].get("event").get("startDateTime")).tz_convert("UTC")

In [ ]:
pd.to_datetime(all_relevant_location_data[200].get("event").get("startDateTime"))

In [ ]:

print(all_relevant_location_data[200].get("event").get("startDateTime"))


In [ ]:
# keep only Dresden and Freiburg

In [ ]:
dt = pd.to_datetime(all_relevant_location_data[100].get("event").get("startDateTime"))

In [ ]:
dt

In [ ]:
type(dt)

In [ ]:
pd.to_datetime("2024-01-01")

## keep only relevant dates

In [ ]:
all_relevant_location_and_date_data = []
start_date_1 = pd.to_datetime("2024-01-01").tz_localize('UTC')
end_date_1 = pd.to_datetime("2024-04-01").tz_localize('UTC')
start_date_2 = pd.to_datetime("2024-09-01").tz_localize('UTC')
end_date_2 = pd.to_datetime("2024-11-01").tz_localize('UTC')

for record in tqdm(all_relevant_location_data):   
    event_start_date = record.get("event").get("startDateTime")
    event_start_date = pd.to_datetime(event_start_date).tz_convert("UTC")

    if start_date_1 <=event_start_date <= end_date_1 or start_date_2 <=event_start_date <= end_date_2:
        all_relevant_location_and_date_data.append(record)
        
            
    
    
    

In [ ]:
len(all_relevant_location_and_date_data)

In [ ]:
with open("apify original DD FB.json", 'w') as json_file:
    json.dump(all_relevant_location_and_date_data, json_file, indent=4) 